In [ ]:
import matplotlib.pyplot as plt
from random import shuffle
import seaborn as sns
import pandas as pd
import numpy as np
import argparse
import errno
import json
import cv2
import os

# Categorización de imágenes por anomalías

In [ ]:
with open(layouts_info_path, "r") as json_file:
    layouts_info = json.load(json_file)

# Initialize a list to hold the paths to all images
dataset_image_paths = []

# Loop through all files in the specified directory
for filename in os.listdir(images_path):
    # Check if the file is an image (optional: adjust the condition based on your image formats, e.g., '.png')
    if filename.endswith('.png'):
        # Create the full path to the image and add it to the list
        full_path = os.path.join(images_path, filename)
        dataset_image_paths.append(full_path)

## Límites por categoría

In [ ]:
anomalías = {
    "brillo": {
        "umbrales": [0.1, 0.25, 0.6, 0.7],
        "etiquetas": ["imágenes_muy_oscuras", "imágenes_oscuras", "imágenes_normales", "imágenes_brillantes", "imágenes_muy_brillantes"],
        "función_puntuación": image_lightness
    },
    "desenfoque_fft": {
        "umbrales": [5.5, 6.1],
        "etiquetas": ["imágenes_muy_desenfocadas", "imágenes_desenfocadas", "imágenes_normales"],
        "función_puntuación": calculate_blur_fft
    },
    "desenfoque_sobel": {
        "umbrales": [40, 80],
        "etiquetas": ["imágenes_muy_desenfocadas", "imágenes_desenfocadas", "imágenes_normales"],
        "función_puntuación": calculate_blur_sobel
    },
    "oscuridad": {
        "umbrales": [2, 5, 15],
        "etiquetas": ["imágenes_normales", "imágenes_con_muy_poco_negro", "imágenes_con_5_20_de_negro", "imágenes_con_mucho_negro"],
        "función_puntuación": calculate_black_pixel_percentage
    },
    "relación_aspecto": {
        "umbrales": [10, 15, 25],
        "etiquetas": ["imágenes_ligeramente_largas", "imágenes_normales", "imágenes_largas", "imágenes_muy_largas"],
        "función_puntuación": image_aspect_ratio
    },
}


## Funciones auxiliares

In [ ]:
def crear_directorio_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise exc

def extraer_id_diseño(ruta_archivo):
    """
    Extrae el ID de diseño del nombre de archivo de imagen encontrado en la ruta de archivo dada.
    """
    # Extraer el nombre del archivo de la ruta del archivo
    nombre_archivo = ruta_archivo.split('/')[-1]

    # Se asume que el formato del nombre de archivo es "layoutId_plotId.png"
    # Dividir el nombre del archivo en los guiones bajos y tomar la primera parte que es el ID de diseño
    id_diseño_str = nombre_archivo.split('_')[0]

    # Convertir el ID de diseño de cadena a entero
    id_diseño = int(id_diseño_str)

    return id_diseño


In [ ]:
def luminosidad_imagen(ruta_imagen):
    """
    Calcula la luminosidad de una imagen mediante el valor RGB máximo. Hay múltiples formas de medir el brillo/luminosidad,
    esta fue subjetivamente la más efectiva para imágenes de madurez.
    :param ruta_imagen: La ruta al archivo de imagen.
    :return: El puntaje de luminosidad de la imagen.
    """
    img = cv2.imread(ruta_imagen)
    img_array = np.array(img, dtype=np.float32)
    nr_de_pixeles = img_array.shape[0] * img_array.shape[1]
    suma_pixeles = np.sum(np.max(img_array, axis=2))
    return suma_pixeles / (255 * float(nr_de_pixeles))


def calcular_desenfoque_fft(ruta_imagen):
    """
    Calcula el puntaje de desenfoque de una imagen utilizando la Transformada Rápida de Fourier (FFT).
    :param ruta_imagen: La ruta al archivo de imagen.
    :return: El puntaje de desenfoque de la imagen.
    """
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    f_transform = np.fft.fft2(imagen)
    f_shift = np.fft.fftshift(f_transform)
    espectro_magnitud = np.log(np.abs(f_shift) + 1)  # Evita log(0)
    puntaje_desenfoque = np.sum(espectro_magnitud) / (imagen.shape[0] * imagen.shape[1])
    return puntaje_desenfoque


def calcular_desenfoque_sobel(ruta_imagen):
    """
    Calcula el puntaje de desenfoque de una imagen utilizando el magnitud del gradiente Sobel.
    :param ruta_imagen: La ruta al archivo de imagen.
    :return: El puntaje de desenfoque de la imagen.
    """
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    sobel_x = cv2.Sobel(imagen, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(imagen, cv2.CV_64F, 0, 1, ksize=3)
    magnitud_gradiente = cv2.magnitude(sobel_x, sobel_y)
    puntaje_desenfoque = np.mean(magnitud_gradiente)
    return puntaje_desenfoque


def calcular_porcentaje_pixeles_negros(ruta_imagen):
    """
    Calcula el porcentaje de pixeles completamente negros en la imagen, no las sombras de los pixeles más oscuros,
    solo los negros completos.
    :param ruta_imagen: La ruta al archivo de imagen.
    :return: El puntaje de negrura completo de la imagen.
    """
    imagen = cv2.imread(ruta_imagen)
    total_pixeles = imagen.shape[0] * imagen.shape[1]
    pixeles_negros = (imagen == [0, 0, 0]).all(axis=2).sum()
    porcentaje = (pixeles_negros / total_pixeles) * 100
    return porcentaje

def categorizar_imagenes_con_umbral(resultados_anomalía_float32, umbrales, etiquetas):
    """
    Categoriza imágenes basándose en umbrales especificados.
    :param resultados_anomalía_float32: Lista de medidas de anomalías de imagen. [(ruta_imagen, medida_anomalía),...]
    :param umbrales: Valores de umbral para la categorización. Por ejemplo: [5.5, 6.1]
    :param etiquetas: Lista de etiquetas para cada categoría. ["imágenes_muy_desenfocadas", "imágenes_desenfocadas", "imágenes_normales"]
    :return: Lista de rutas de imágenes ordenadas correspondientes a cada etiqueta.
    """
    categorías = [[] for _ in range(len(umbrales) + 1)]

    for valor_imagen, puntaje_anomalía in resultados_anomalía_float32:
        for i, umbral in enumerate(umbrales):
            if puntaje_anomalía <= umbral:
                categorías[i].append(valor_imagen)
                break
        else:
            categorías[-1].append(valor_imagen)

    for etiqueta, categoría in zip(etiquetas, categorías):
        print(f"{etiqueta}\t-> {len(categoría)}")

    return categorías


def relación_aspecto_imagen(ruta_imagen):
    imagen = cv2.imread(ruta_imagen)
    return np.shape(imagen)[1] / np.shape(imagen)[0]


def procesar_imágenes_de_anomalías(rutas_imágenes_dataset, nombre_anomalía, etiquetas_umbral,
                                   valores_umbral, función_medición, ruta_salida):
    """
    Procesa y categoriza imágenes de anomalías. Crea un gráfico de distribución de la anomalía.
    :param nombre_anomalía: Nombre de la anomalía.
    :param etiquetas_umbral: Lista de etiquetas para los niveles de umbral.
    :param valores_umbral: Lista de valores de umbral.
    :param función_medición: Función para medir anomalías.
    :param ruta_salida: Ruta para guardar resultados y visualizaciones.
    :return: Lista de imágenes categorizadas.
    """
    resultados = []
    ruta_archivo = os.path.join(ruta_salida, f"{nombre_anomalía.lower().replace(' ', '_')}_resultados.json")

   


## Funciones gráficos

In [ ]:
def guardar_grafico_y_limpiar_memoria(ruta_guardado, nombre_archivo):
    mkdir_p(ruta_guardado)
    plt.savefig(os.path.join(ruta_guardado, nombre_archivo))
    plt.cla()
    plt.close()


def distribucion_medicion_anomalía(resultados_medicion, nombre_anomalia, directorio_guardado, xlim=None):
    mediciones = [entrada[1] for entrada in resultados_medicion]

    plt.figure(figsize=(10, 6))
    sns.histplot(data=mediciones, bins=15, kde=True, color='blue')

    plt.title(f"Distribución de {nombre_anomalia}", fontsize=16)
    plt.xlabel(f"{nombre_anomalia}", fontsize=14)
    plt.ylabel("Número de imágenes", fontsize=14)
    if xlim:
        plt.xlim(xlim[0], xlim[1])

    ruta_guardado = os.path.join(directorio_guardado, nombre_anomalia)
    guardar_grafico_y_limpiar_memoria(ruta_guardado, f"{nombre_anomalia}_distribucion_medicion.jpg")


def cuadricula_intervalos_anomalias(nombre_anomalia, listas_columnas, nombres_columnas, ruta_directorio_imagenes, directorio_guardado):
    for id_grado, grado in enumerate(listas_columnas):
        shuffle(listas_columnas[id_grado])

    num_filas = 40
    fig, axes = plt.subplots(num_filas, len(listas_columnas), figsize=(50, 80))

    for fila in range(num_filas):
        for grado in range(len(listas_columnas)):
            if len(listas_columnas[grado]) > (fila + 1):
                ruta_imagen = listas_columnas[grado][fila]
            else:
                break
            imagen = cv2.imread(os.path.join(ruta_directorio_imagenes, ruta_imagen))
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
            axes[fila, grado].imshow(imagen)
            axes[fila, grado].text(0.5, -0.15, ruta_imagen, size=15, ha="center", transform=axes[fila, grado].transAxes)
            axes[fila, grado].axis('off')

            # Obtener la altura y el tamaño de la imagen
            altura, ancho, _ = imagen.shape
            texto_tamaño = f"Tamaño: {ancho}x{altura}"
            axes[fila, grado].text(
                0.5, 1.05, texto_tamaño, size=15, ha="center", transform=axes[fila, grado].transAxes
            )

    for ax, col in zip(axes[0], nombres_columnas):
        ax.set_title(col.replace('_', ' '), fontsize=60, y=1.4, fontweight="bold")

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.suptitle(f"Ejemplos de {nombre_anomalia}", fontsize=80, fontweight="bold", y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    ruta_guardado = os.path.join(directorio_guardado, nombre_anomalia)
    guardar_grafico_y_limpiar_memoria(ruta_guardado, f"{nombre_anomalia.lower().replace(' ', '_')}_particion_intervalos.jpg")


# Procesar cada imagen para cada anomalía y calcular sus grados
grados_anomalias = {}
for nombre_an, datos_anomalia in anomalías.items():
    grados_anomalias[nombre_an] = procesar_imagenes_anomalía(rutas_imagenes_conjunto_datos, nombre_an, datos_anomalia["etiquetas"], datos_anomalia["umbrales"],
                                                             datos_anomalia["función_puntuación"], directorio_salida)
    cuadricula_intervalos_anomalias(nombre_an, grados_anomalias[nombre_an], datos_anomalia["etiquetas"],
                                    ruta_imágenes, directorio_guardado=directorio_salida)


In [ ]:
def cargar_y_categorizar_datos_de_anomalía(nombre_anomalía, ruta_valores_anomalías, anomalías):
    with open(os.path.join(ruta_valores_anomalías, f"{nombre_anomalía}_resultados.json"), 'r') as archivo:
        datos = json.load(archivo)

    umbrales = anomalías[nombre_anomalía]['umbrales']
    etiquetas = anomalías[nombre_anomalía]['etiquetas']
    datos_categorizados = {etiqueta: [] for etiqueta in etiquetas}

    for ruta_imagen, valor in datos:
        indice_categoría = sum([valor > umbral for umbral in umbrales])
        datos_categorizados[etiquetas[indice_categoría]].append(ruta_imagen)

    return datos_categorizados


In [ ]:
def graficar_datos_de_anomalía_por_metadatos(datos_anomalía, layouts_info, nombre_anomalía, ruta_salida, campo_metadatos='country'):
    # Inicializar diccionario para almacenar los datos de conteo
    conteos_metadatos = {}
    for info in layouts_info:
        if info[campo_metadatos] not in conteos_metadatos:
            conteos_metadatos[info[campo_metadatos]] = {etiqueta: 0 for etiqueta in datos_anomalía.keys()}

        for etiqueta, imágenes in datos_anomalía.items():
            for imagen in imágenes:
                try:
                    # Extraer layout_id del nombre de archivo
                    nombre_archivo = imagen.split('/')[-1]  # Obtener la parte del nombre de archivo después del último '/'
                    id_diseño = int(nombre_archivo.split('_')[0])  # Obtener la parte antes del primer '_' y convertir a entero
                    if info['layout_id'] == id_diseño:
                        conteos_metadatos[info[campo_metadatos]][etiqueta] += 1
                except ValueError:
                    # Saltar archivos donde id_diseño no es un entero
                    continue

    # Asegurar que exista el directorio de salida
    directorio_salida = os.path.join(ruta_salida, nombre_anomalía, f"por_{campo_metadatos}")
    os.makedirs(directorio_salida, exist_ok=True)

    # Graficar y guardar las figuras
    for metadato, conteos in conteos_metadatos.items():
        etiquetas = list(conteos.keys())
        valores = list(conteos.values())

        plt.figure(figsize=(10, 6))
        ax = sns.barplot(x=etiquetas, y=valores, palette="viridis")
        plt.title(f"Distribución de {nombre_anomalía} en {metadato}")
        plt.xlabel("Grado de Anomalía")
        plt.ylabel("Número de Imágenes")
        plt.xticks(rotation=15)

        # Agregar etiquetas de texto encima de las barras
        for i, valor in enumerate(valores):
            ax.text(i, valor, f'{valor}', color='black', ha='center', va='bottom')

        # Guardar la gráfica
        ruta_archivo_guardado = os.path.join(directorio_salida, f"{metadato.replace(' ', '_').lower()}.png")
        plt.savefig(ruta_archivo_guardado)
        plt.close()


In [ ]:
nombres_anomalías = list(anomalías.keys())  # Todas las anomalías en la lista de anomalías
nombres_anomalías = [s[0].lower() + s[1:] if s else "" for s in nombres_anomalías]

for nombre_anomalía in nombres_anomalías:
    datos_anomalía = cargar_y_categorizar_datos_anomalía(nombre_anomalía, output_path, anomalías)
    plot_anomaly_data_by_metadata(datos_anomalía, información_diseños, nombre_anomalía, output_path, metadata_field='país')
